# 随机神经网络(RaNN)的先验理论知识

### 随机向量函数链接（RVFL）网络
随机向量函数链接（Random Vector Functional-Link, RVFL）是一种经典的单隐藏层神经网络架构，最早由Pao在1994年提出，并由Igelnik与Pao在1995年进一步完善。其核心特点是**隐藏层参数完全随机生成，且在整个训练过程中保持固定**，这一设计使其区别于传统的神经网络。


#### RVFL的数学表达
RVFL的输出可以表示为：
$$u_M(x) = \sum_{j=1}^M \beta_j \sigma\left(w_j^\top x + b_j\right)\tag{9.1}$$
其中：
- $x$是输入样本；
- $w_j$（隐藏层权重）、$b_j$（隐藏层偏置）是从预设分布（如高斯分布、均匀分布）中随机采样得到的参数，采样后永久固定；
- $\sigma(\cdot)$是激活函数（如Sigmoid、ReLU）；
- $\beta_j$是输出层的系数；
- $\rho$代表$w_j$与$b_j$的采样分布。


#### RVFL的核心组件
1. **随机隐藏层**
   隐藏层的每个神经元输出定义为：
   $$h_j(x) = \sigma\left(w_j^\top x + b_j\right) \quad (j = 1,2,...,M)\tag{9.2}$$
   这里的$w_j$和$b_j$仅在网络初始化时采样一次，后续不会通过训练调整——这是RVFL最关键的设计，直接简化了训练流程。


2. **可训练的输出层**
   RVFL的训练仅针对输出层的系数$\beta_j$进行，通常采用最小二乘法求解最优值：
   $$\beta^* = \arg \min_\beta \left\|\Phi\beta - y\right\|_2^2 \tag{9.3}$$
   其中$\Phi$是隐藏层输出矩阵（每行对应一个样本的隐藏层输出），$y$是真实标签。由于$\Phi$是固定的，这个优化问题是凸问题，可以直接通过矩阵运算（如伪逆）快速求解，无需迭代。


3. **无需反向传播**
   传统神经网络依赖反向传播算法迭代更新所有层的参数，而RVFL的隐藏层参数固定，仅优化输出层的线性系数——这使得RVFL的训练过程无迭代、无梯度计算，训练速度远快于传统网络。


#### RVFL的本质与地位
从本质上看，RVFL可以理解为随机非线性基函数 + 线性回归的组合：隐藏层的随机参数相当于构造了一组非线性的基函数，而输出层的训练则是在这组基函数的基础上做线性拟合。

作为早期的“随机化神经网络”，RVFL是极限学习机（ELM）和随机特征方法的重要先驱，其“随机化隐藏层+线性输出层”的设计思路，为后续高效、轻量化的神经网络模型提供了核心启发。

### 极限学习机（ELM）：随机化神经网络的高效变体
极限学习机（Extreme Learning Machine，ELM）是随机化神经网络家族中的经典特殊形式，由Huang等人于2006年提出。它继承了随机化隐藏层+线性输出层的核心设计，但进一步简化了训练流程，是轻量化、高速训练模型的典型代表。


#### ELM的数学形式
ELM的输出可表示为：
$$u_\theta(x) = \sum_{j=1}^M \beta_j \sigma\left(w_j^\top x + b_j\right) \tag{9.4}$$
其中：
- $x$为输入样本；
- $w_j$（隐藏层权重）、$b_j$（隐藏层偏置）仅在网络初始化时**随机采样一次**，后续训练过程中**永久固定**（无需任何调整）；
- $\sigma(\cdot)$是隐藏层的激活函数（如Sigmoid、ReLU等非线性函数）；
- $\beta_j$是输出层的线性系数，是ELM中**唯一需要训练的参数**。


#### ELM的训练：仅需求解线性系统
与传统神经网络的迭代式训练不同，ELM的训练过程等价于求解一个线性方程组：
我们先通过固定的隐藏层参数，将所有输入样本转换为“隐藏层输出矩阵”$\Phi$（$\Phi$的每一行对应一个样本的隐藏层输出向量）；再以$\Phi$为基、真实标签$y$为目标，通过最小二乘法优化输出层系数$\beta$：
$$\beta^* = \arg \min_\beta \left\|\Phi\beta - y\right\|_2^2 \tag{9.5}$$

由于$\Phi$是固定的，这个优化问题是凸的，可直接通过矩阵运算得到解析解：
$$\beta^* = \left(\Phi^\top \Phi\right)^{-1}\Phi^\top y \tag{9.6}$$


#### ELM的本质与核心优势
ELM的本质可以概括为：随机非线性特征映射 + 线性回归——隐藏层的随机参数负责将输入映射到高维非线性特征空间，输出层则在这个空间中做简单的线性拟合。

其核心优势在于：
1. **训练速度极快**：无需反向传播、无需迭代，仅需一次矩阵运算即可完成训练；
2. **实现简单**：无需调优隐藏层参数，仅需设置隐藏层神经元数量；
3. **泛化能力强**：随机化隐藏层降低了过拟合风险，在不少任务中能达到与传统网络相当的精度。

### 蒙特卡罗近似：随机化神经网络的逼近理论
**核心目标**：揭示单隐藏层随机化神经网络的逼近能力——当隐藏层参数（如权重$w_j$、偏置$b_j$）是随机采样得到的，随着隐藏层神经元数量$M$趋于无穷大，该网络可以在$L^2$空间的度量下，收敛到任意目标函数$f(x)$。

其对应的网络输出形式为：
$$u_M(x) = \frac{1}{M} \sum_{j=1}^M \alpha_j \sigma\left(w_j^\top x + b_j\right)\tag{9.7}$$


#### 前提假设：目标函数的积分表示
要证明上述逼近性，首先假设**目标函数$f(x)$可以表示为积分形式**：
设$f: \mathcal{X} \to \mathbb{R}$（例如输入空间$\mathcal{X}$是$d$维实数空间$\mathbb{R}^d$的子集），满足：
$$f(x) = \int_\Omega a(\omega) \phi(x, \omega) p(\omega) d\omega \tag{9.8}$$

其中各符号的含义是：
- $\omega \in \Omega$：参数向量（在随机化神经网络中，$\omega$对应隐藏层的参数组合$(w, b)$）；
- $p(\omega)$：$\Omega$上的概率密度函数（即隐藏层参数的采样分布）；
- $\phi(x, \omega)$：非线性基函数（对应神经网络中隐藏层的计算，例如$\phi(x, \omega) = \sigma(w^\top x + b)$，$\sigma$是激活函数）；
- $a(\omega)$：平方可积的系数函数（后续会对应网络输出层的系数）。


#### 随机化网络与蒙特卡罗近似的关联
蒙特卡罗方法的核心是“用随机采样的均值近似积分”——而随机化神经网络的设计，恰好是这一思想的体现：

从概率密度$p(\omega)$中**独立同分布地抽取$M$个参数样本$\{\omega_j\}_{j=1}^M$**，然后将积分形式的目标函数$f(x)$，替换为“样本加权和的均值”，即得到随机化网络的输出：
$$u_M(x) = \frac{1}{M} \sum_{j=1}^M a(\omega_j) \phi(x, \omega_j) \tag{9.9}$$

当$M$足够大时，根据**大数定律**，样本均值 $\frac{1}{M}\sum_{j=1}^M a(\omega_j)\phi(x, \omega_j)$ 会收敛到积分 $\int_\Omega a(\omega)\phi(x, \omega)p(\omega)d\omega$（即目标函数 $f(x)$）。

这就解释了：为什么随机化神经网络（如RVFL、ELM）的隐藏层参数只需随机采样、无需训练——因为其本质是用“蒙特卡罗采样”来近似目标函数的积分表示，只要采样数量足够多，网络就能自然逼近目标函数。


均方收敛性是验证 “随机化神经网络能逼近目标函数” 的核心理论依据，以下是其详细推导与解读：

#### 1. 随机估计器的定义
我们先定义随机化神经网络对应的**随机估计器**：
$$u_M(x) = \frac{1}{M} \sum_{j=1}^M a(\omega_j) \phi(x, \omega_j) \tag{9.10}$$
其中，$\omega_j$是从概率密度$p(\omega)$中**独立同分布（i.i.d.）采样**的参数（对应隐藏层的权重、偏置组合）。


#### 2. 估计器与目标函数的关联
对任意输入$x$，$u_M(x)$本质是**蒙特卡罗近似**——它逼近的积分恰好是目标函数$f(x)$：
$$f(x) = \int_\Omega a(\omega) \phi(x, \omega) p(\omega) d\omega = \mathbb{E}_\omega\left[a(\omega)\phi(x, \omega)\right]  \tag{9.11}$$
这里$\mathbb{E}_\omega[\cdot]$表示对参数$\omega$的期望，也就是说：**目标函数$f(x)$是随机变量$a(\omega)\phi(x, \omega)$的期望**。


#### 3. 无偏性：估计器的期望等于目标函数
$u_M(x)$是$f(x)$的**无偏估计**——即估计器的期望恰好等于目标函数（逐点成立）：
$$\mathbb{E}_{\omega_1, \dots, \omega_M}\left[u_M(x)\right] = \frac{1}{M} \sum_{j=1}^M \mathbb{E}_{\omega_j}\left[a(\omega_j)\phi(x, \omega_j)\right] \tag{9.12}$$
由于$\omega_j$是独立同分布的，每个$\mathbb{E}_{\omega_j}\left[a(\omega_j)\phi(x, \omega_j)\right]$都等于$\mathbb{E}_\omega\left[a(\omega)\phi(x, \omega)\right]$，因此上式最终等于$f(x)$。


#### 4. 方差界：估计器的波动随$M$减小
为分析估计器的波动，定义随机变量$Z(x, \omega) = a(\omega)\phi(x, \omega)$，则$u_M(x)$可表示为$Z(x, \omega_j)$的均值：
$$u_M(x) = \frac{1}{M}\sum_{j=1}^M Z(x, \omega_j) \tag{9.13}$$

若假设$Z(x, \omega)$的**二阶矩有限**（即$\mathbb{E}\left[Z(x, \omega)^2\right] < \infty$，这是一个温和的条件），则$u_M(x)$的方差满足：
$$\text{Var}\left(u_M(x)\right) = \frac{1}{M} \text{Var}\left(Z(x, \omega)\right) \leq \frac{C(x)}{M} \tag{9.14}$$
其中$C(x)$是与$x$相关的有限常数。这意味着：**随着隐藏层神经元数量$M$增大，估计器的方差会以 $\frac{1}{M}$ 的速率减小**。


#### 5. 均方收敛：估计器逼近目标函数
均方误差是衡量估计器精度的核心指标，对$u_M(x)$而言：
$$\mathbb{E}\left[\left(u_M(x) - f(x)\right)^2\right] = \text{Var}\left(u_M(x)\right) \tag{9.15}$$

结合方差界的结论，当$M \to \infty$时：
$$\mathbb{E}\left[\left(u_M(x) - f(x)\right)^2\right] \leq \frac{C(x)}{M} \xrightarrow{M \to \infty} 0 \tag{9.16}$$

因此，随机化神经网络的输出 $u_M(x)$ ，会在**均方意义下逐点收敛于目标函数$f(x)$**——这从理论上保证了：只要隐藏层神经元数量足够多，随机化神经网络就能以任意精度逼近目标函数。

此前我们证明了随机化神经网络的**逐点均方收敛性**（即对单个输入$x$，$u_M(x)$能逼近$f(x)$），这里要进一步推广到**输入域$\mathcal{X}$上的全局$L^2$逼近**——这是“万能逼近”的核心理论支撑。


#### 1. $L^2$全局误差的定义
为衡量整个输入域$\mathcal{X}$（配备测度$\mu$）上的逼近效果，我们用$L^2$范数定义全局误差：
$$\left\|u_M - f\right\|_{L^2(\mathcal{X})}^2 = \int_{\mathcal{X}} \left|u_M(x) - f(x)\right|^2 d\mu(x) \tag{9.17}$$
它表示$u_M$与$f$在整个输入域上的“平方误差积分”，能反映全局的逼近精度。


#### 2. $L^2$均方误差的转化
我们需要分析 $L^2$误差的期望（即 **$L^2$均方误差**）：
$$\mathbb{E}\left[\left\|u_M - f\right\|_{L^2}^2\right] = \mathbb{E}\left[ \int_{\mathcal{X}} \left|u_M(x) - f(x)\right|^2 d\mu(x) \right]  \tag{9.18}$$

利用**富比尼定理**（在积分与期望满足绝对可积的条件下），**交换“期望”与“积分”的顺序**：
$$\mathbb{E}\left[\left\|u_M - f\right\|_{L^2}^2\right] = \int_{\mathcal{X}} \mathbb{E}\left[ \left|u_M(x) - f(x)\right|^2 \right] d\mu(x) \tag{9.19}$$

这一步将“全局$L^2$均方误差”转化为了“逐点均方误差在输入域上的积分”，从而可以复用之前的逐点收敛结论。


#### 3. 结合逐点收敛的结论推导全局界
由之前的逐点均方收敛分析，我们已知：对任意$x \in \mathcal{X}$，
$$\mathbb{E}\left[ \left|u_M(x) - f(x)\right|^2 \right] \leq \frac{C(x)}{M} \tag{9.20}$$
其中$C(x)$是与$x$相关的有限常数。

若进一步假设$C(x)$在$\mathcal{X}$上关于测度$\mu$**可积**（即$\int_{\mathcal{X}} C(x) d\mu(x) < \infty$，这一条件对应$a(\omega)\phi(x, \omega)$是平方可积函数），则全局$L^2$均方误差满足：
$$\mathbb{E}\left[\left\|u_M - f\right\|_{L^2}^2\right] \leq \frac{1}{M} \int_{\mathcal{X}} C(x) d\mu(x) = \frac{C}{M} \tag{9.21}$$
这里$C$是与$M$无关的有限常数。



因此，当隐藏层神经元数量$M \to \infty$时，全局$L^2$均方误差会趋于0：
$$\mathbb{E}\left[\left\|u_M - f\right\|_{L^2}^2\right] \leq \frac{C}{M} \xrightarrow{M \to \infty} 0 \tag{9.22}$$


这意味着：**随机化神经网络能在$L^2$均方意义下全局逼近目标函数$f$**，且其$L^2$范数意义下的收敛速率为**蒙特卡罗速率$\mathcal{O}(1/\sqrt{M})$**（即误差随$M$的平方根倒数衰减）。


---

### 以蒙特卡罗估计器为核心的万能逼近：RVFL/ELM的理论本质
随机化神经网络（如RVFL、ELM）的“万能逼近能力”，本质是**蒙特卡罗估计器逼近积分的直接结果**。以下是其完整逻辑与理论推导：


#### RVFL网络的基本形式
RVFL网络的输出可表示为：
$$u_M(x) = \frac{1}{M} \sum_{j=1}^M \alpha_j \sigma\left(w_j^\top x + b_j\right) \tag{9.23}$$
其中：
- $(w_j, b_j)$：隐藏层的权重与偏置，从分布$p(w, b)$中**独立同分布（i.i.d.）随机采样一次后固定**；
- $\alpha_j$：输出层的可训练权重（是网络中唯一需要优化的参数）；
- $\sigma(\cdot)$：隐藏层的激活函数（如Sigmoid、ReLU）。


#### 目标函数的积分表示假设
要证明RVFL的万能逼近性，首先假设**目标函数$f(x)$可以表示为积分形式**：
$$f(x) = \int a(w,b) \sigma\left(w^\top x + b\right) p(w,b) \, dw db \tag{9.24}$$

这个假设的合理性在于：许多实际任务中的目标函数，都可以通过“非线性基函数（$\sigma(w^\top x + b)$）的加权积分”来表示，而$p(w,b)$是基函数参数的分布，$a(w,b)$是对应的权重系数。


#### 随机特征的定义
为衔接蒙特卡罗估计，我们将积分中的参数统一表示为$\omega = (w,b)$，并定义：
- **随机特征**：$\phi(x,\omega) := \sigma\left(w^\top x + b\right)$（即隐藏层神经元的输出，由随机参数$\omega$决定）；
- **系数函数**：$a(\omega) = a(w,b)$（对应积分中的权重系数）。

此时，目标函数的积分形式可简化为：
$$f(x) = \int a(\omega) \phi(x,\omega) p(\omega) \, d\omega \tag{9.25}$$


#### RVFL的本质：蒙特卡罗估计器
RVFL的输出$u_M(x)$，实际上是对上述积分的**蒙特卡罗估计**：
从分布$p(\omega)$中独立同分布采样$M$个参数$\{\omega_j\}_{j=1}^M$，用“样本加权和的均值”逼近积分：
$$u_M(x) = \frac{1}{M} \sum_{j=1}^M a(\omega_j) \phi(x,\omega_j), \quad \omega_j \sim p(\omega) \tag{9.26}$$

这里的$a(\omega_j)$对应RVFL中可训练的输出层权重$\alpha_j$——也就是说，RVFL的训练过程，本质是在“随机采样的基函数”上拟合积分的权重系数。


#### 基于蒙特卡罗的收敛性保证
结合此前的理论，蒙特卡罗估计的$L^2$均方误差满足：
$$\mathbb{E}\left[\left\|u_M - f\right\|_{L^2}^2\right] \leq \frac{C}{M} \xrightarrow{M \to \infty} 0 \tag{9.27}$$

其中$C$是与$M$无关的有限常数，这意味着：**随着隐藏层神经元数量$M$增大，RVFL的输出会以$L^2$均方意义收敛到目标函数$f(x)$**。


#### 万能逼近的结论（蒙特卡罗式）
对任意给定的精度$\varepsilon > 0$，只需选择足够大的$M$，就能保证：
$$\mathbb{E}\left[\left\|u_M - f\right\|_{L^2}\right] < \varepsilon \tag{9.28}$$

这就是“万能逼近”的核心：**以极高的概率，随机采样得到的隐藏层参数$(w_j, b_j)$，能让RVFL的输出与目标函数的$L^2$误差任意小**——这也解释了为什么RVFL/ELM无需训练隐藏层参数，却能具备强大的逼近能力。

---

### 再生核希尔伯特空间（RKHS）
再生核希尔伯特空间（Reproducing Kernel Hilbert Space，简称RKHS）是**函数构成的希尔伯特空间**，其核心特征是与一个“再生核函数”绑定，从而将“函数运算”转化为更易处理的线性代数操作。


#### RKHS的严格定义
设$\mathcal{H}$是**定义在输入域$\mathcal{X}$上的实值函数集合**，且$\mathcal{H}$满足希尔伯特空间的基本性质（完备性、配备内积$\langle \cdot, \cdot \rangle_{\mathcal{H}}$）。若存在一个**对称正定核函数**
$$k: \mathcal{X} \times \mathcal{X} \to \mathbb{R}\tag{9.29}$$
同时满足以下两个条件，则$\mathcal{H}$被称为**再生核希尔伯特空间（RKHS）**：


##### 条件1：核函数诱导的函数属于空间
对任意固定的输入$x \in \mathcal{X}$，将$k$的第二个参数固定为$x$后得到的函数$k(\cdot, x)$（即关于第一个参数的函数），是$\mathcal{H}$中的元素：
$$k(\cdot, x) \in \mathcal{H}, \quad \forall x \in \mathcal{X}\tag{9.30}$$


##### 条件2：再生性（核心性质）
对$\mathcal{H}$中的**任意函数$f$**，函数在点$x$处的取值$f(x)$，等于$f$与$k(\cdot, x)$在$\mathcal{H}$中的内积：
$$f(x) = \langle f, k(\cdot, x) \rangle_{\mathcal{H}}, \quad \forall f \in \mathcal{H}, \forall x \in \mathcal{X}\tag{9.31}$$


##### RKHS的直观解释
再生性是RKHS最关键的性质，可以从三个角度理解：
1. **特征向量的类比**：$k(\cdot, x)$可看作“输入$x$在$\mathcal{H}$这个函数空间中对应的特征元素”（类似有限维空间中的特征向量）；
2. **函数求值=内积运算**：计算函数$f$在$x$处的取值$f(x)$，无需直接计算函数本身，只需计算$f$与$x$对应的“特征元素”$k(\cdot, x)$的内积；
3. **线性代数的统一**：RKHS把“函数求值”这一非线性操作，转化为了希尔伯特空间中的内积（线性代数运算），让函数的处理可以复用线性空间的理论工具。


#### RKHS的核心本质
从定义可直接推导出：**核函数$k(x, x')$本质是$k(\cdot, x)$与$k(\cdot, x')$在$\mathcal{H}$中的内积**，即
$$k(x, x') = \langle k(\cdot, x), k(\cdot, x') \rangle_{\mathcal{H}}\tag{9.32}$$

这也呼应了核方法的核心思想：**核函数是无穷维特征空间中的内积**——而RKHS正是这个“无穷维特征空间”的数学载体。

#### RKHS及其对应的核函数示例

##### 1. 线性核
核函数形式：
$$k(x, x') = x^\top x' \tag{9.33}$$
对应的RKHS：配备点积内积的$\mathbb{R}^d$空间（即$d$维欧氏空间）。


##### 2. 多项式核
核函数形式：
$$k(x, x') = \left(1 + x^\top x'\right)^p \tag{9.34}$$
对应的RKHS：**次数不超过$p$的所有多项式构成的空间**。


##### 3. 高斯（RBF）核
核函数形式：
$$k(x, x') = \exp\left(-\frac{\|x - x'\|^2}{2\sigma^2}\right) \tag{9.35}$$
对应的RKHS：**无穷维的光滑函数空间**。


##### 4. 拉普拉斯核
核函数形式：
$$k(x, x') = \exp\left(-\|x - x'\|_1/\sigma\right) \tag{9.36}$$
对应的RKHS：**频率呈指数衰减的函数空间**。


不同核函数对应不同的函数空间——选择核函数，本质就是选择模型的假设空间。


#### 核方法：能力强大却受限于计算成本
核方法（如核回归、支持向量机）的模型形式为：
$$f(x) = \sum_{i=1}^N \alpha_i k(x, x_i) \tag{9.37}$$
它是机器学习中表达能力极强的方法，但**计算复杂度高**是其规模化应用的最大障碍。


核方法的优势源于其背后的RKHS理论，具体体现在：
1. **假设空间丰富**：RKHS能容纳复杂的函数形式，远超过普通线性模型的拟合范围；
2. **无穷维特征映射**：核函数对应的特征映射$\phi(x)$可以是无穷维的（如高斯核），无需显式升维就能捕捉非线性关系；
3. **理论保障完备**：既具备“万能逼近”能力，又有严格的统计误差界，泛化性能更可靠。


核方法的所有运算都依赖**核矩阵**（也叫Gram矩阵）：
$$K_{ij} = k(x_i, x_j) \in \mathbb{R}^{N \times N} \tag{9.38}$$
（$N$是训练样本数量）

这直接导致了难以承受的成本：
- **训练成本**：矩阵求逆、分解等操作的复杂度为$O(N^3)$——当$N$达到万级时，计算量会呈指数级爆炸；
- **存储成本**：核矩阵需要$O(N^2)$的存储空间——若$N=10^4$，仅存储核矩阵就需要约1GB内存（按浮点数计算）；
- **预测成本**：对单个测试样本，需要计算它与所有训练样本的核函数值，复杂度为$O(N)$——在实时预测场景中完全无法满足效率要求。


核方法的“强表达能力”（来自无穷维特征映射），与“高计算成本”（来自核矩阵的稠密性）是一对核心矛盾：
尽管无穷维特征映射让模型能拟合复杂任务，但依赖完整核矩阵的运算方式，使得核方法在 大数据集（$N$较大）上几乎无法落地。

解决这一矛盾的关键，是将“无穷维特征映射$\phi(x)$”**逼近为有限维的随机特征**——通过随机采样得到低维特征映射，替代原有的核矩阵运算，从而将核方法的复杂度从$O(N^3)$降至与样本数量线性相关的级别，实现规模化应用。

### 博赫纳定理：随机傅里叶特征的理论基石
博赫纳定理是连接“平移不变核函数”与“随机特征”的核心理论，它将核函数转化为**随机特征的期望**，为核方法的规模化（如随机傅里叶特征）提供了数学依据。


#### 适用场景：平移不变核
首先明确“平移不变核”的定义：若核函数满足 $k(x, x') = k(x - x')$（即核函数的值仅由输入的差决定），则称其为平移不变核（典型例子：高斯核、拉普拉斯核）。


#### 博赫纳定理的核心结论
对于**连续、正定、平移不变的核函数**，博赫纳定理指出：它可以表示为**傅里叶积分形式**：
$$k(x - x') = \int_{\mathbb{R}^d} e^{i\omega^\top (x - x')} p(\omega) d\omega \tag{9.39}$$

其中：
- $e^{i\omega^\top (x - x')}$是复指数函数（傅里叶基）；
- $p(\omega)$是$\mathbb{R}^d$上的概率密度函数，也称为核的“谱测度”（描述核在频率域的分布）。


#### 从傅里叶积分到随机特征的期望
利用三角恒等式，复指数形式的积分可以转化为**实值随机特征的期望**：
$$k(x,x') = \mathbb{E}_{\omega \sim p(\omega)} \left[ \cos(\omega^\top x + b) \cos(\omega^\top x' + b) \right] \tag{9.40}$$

这里的关键设定是：
- $\omega$从谱测度$p(\omega)$中采样；
- $b$服从$[0, 2\pi]$上的均匀分布（用于引入随机相位，保证特征的无偏性）。


#### 定理的直观意义
博赫纳定理的核心价值在于**将核函数“拆解”为随机特征的期望**：
1. 核函数不再是抽象的内积，而是“随机非线性特征（$\cos(\omega^\top x + b)$）两两内积的期望”；
2. 这天然提供了**蒙特卡罗逼近思路**：用“有限个随机特征的均值”替代“期望”，即可得到核函数的近似。

博赫纳定理是**随机傅里叶特征（Random Fourier Features）** 的数学基础：
通过从$p(\omega)$中采样$M$个随机频率$\omega_1, \dots, \omega_M$，并生成对应的随机特征$\cos(\omega_j^\top x + b_j)$，就能用有限维特征映射近似原核函数——这正是核方法规模化的关键：将无穷维特征映射转化为有限维随机特征，同时保证核函数的近似精度。


### 随机特征：核函数的有限维逼近方法
随机特征（Random Features，RF）是通过**蒙特卡罗采样**将核函数的“无穷维特征映射”转化为“有限维显式特征”的技术，核心是用“样本均值”替代“期望”来逼近核函数。


#### 核心思路：蒙特卡罗采样替代期望
核函数的期望形式（由博赫纳定理推导）可以通过**采样有限个随机参数**来近似——这是随机特征的理论基础。


#### 随机特征的构造步骤
步骤1：采样随机参数

从核函数对应的谱测度$p(\omega)$中采样$M$个频率$\omega_j$，同时从$[0, 2\pi]$均匀分布中采样$M$个随机相位$b_j$：
$$\omega_j \sim p(\omega), \quad b_j \sim \text{Unif}[0, 2\pi]\tag{9.41}$$


步骤2：定义$M$维随机特征映射

构造**显式的$M$维特征向量**（加入归一化因子$\sqrt{\frac{2}{M}}$，保证特征内积的期望等于原核函数）：
$$z(x) = \sqrt{\frac{2}{M}} \begin{bmatrix} \cos(\omega_1^\top x + b_1) \\ \cos(\omega_2^\top x + b_2) \\ \vdots \\ \cos(\omega_M^\top x + b_M) \end{bmatrix} \tag{9.42}$$


步骤3：用特征点积逼近核函数

原核函数$k(x, x')$可由随机特征的点积近似：
$$k(x, x') \approx z(x)^\top z(x') \tag{9.43}$$


#### 随机特征的核心优势
随机特征彻底解决了核方法的规模化问题，具体体现在：
- **维度从无穷到有限**：用显式$M$维特征替代了原核函数的无穷维特征映射$\phi(x)$；
- **模型从核方法到线性方法**：核回归、SVM等核方法，转化为$z(x)$空间中的线性回归/线性分类；
- **复杂度大幅降低**：训练成本从核方法的$O(N^3)$（$N$为样本数）降至$O(NM)$（$M \ll N$），存储与预测成本也同步线性化。


随机特征通过“蒙特卡罗采样+有限维特征构造”，将核函数的无穷维内积转化为有限维特征的点积，既保留了核方法的表达能力，又让其能在大数据集上高效运行——这也是核方法与随机化神经网络的重要连接点。


### 随机特征的逼近保证：理论精度与收敛性
随机特征对核函数的逼近并非“经验性近似”，而是有严格的理论误差保证——这些保证明确了“特征维度$M$”与“逼近精度”的量化关系。


#### 基本设定
用随机特征构造的核函数逼近形式为：
$$k_M(x, x') = z(x)^\top z(x') \tag{9.44}$$
其中$z(x)$是$M$维随机特征映射。


#### 蒙特卡罗集中性保证：期望误差的衰减速率
随机特征逼近的**期望误差**满足：
$$\mathbb{E}\left[ \left| k_M(x, x') - k(x, x') \right| \right] = \mathcal{O}\left( \frac{1}{\sqrt{M}} \right)\tag{9.45}$$

这意味着：
- 误差随$M$的平方根倒数衰减（即“蒙特卡罗速率”）；
- 当$M$增大时，期望误差会稳定减小。


#### 一致界：全局误差的概率保证
对**紧致输入域$\mathcal{X}$**（例如有界的欧氏空间子集），随机特征的**全局最大误差**满足概率界：
$$\Pr\left( \sup_{x,x' \in \mathcal{X}} \left| k_M(x, x') - k(x, x') \right| > \varepsilon \right) \leq \mathcal{O}\left( \frac{1}{\varepsilon^2} \exp(-cM\varepsilon^2) \right) \tag{9.46}$$

其中$c$是与核函数相关的常数。这个界的意义是：
- 全局误差超过$\varepsilon$的概率，随$M$增大呈**指数级衰减**；
- 只需选择足够大的$M$，就能以极高的概率保证全局逼近精度。

这些理论保证带来了三个关键推论：
1. **精度可控**：通过调整$M$的大小，可以灵活控制逼近精度（$M$越大，精度越高）；
2. **解的收敛性**：基于随机特征训练的核模型（如核回归），其解会收敛到“精确核方法”的解；
3. **理论可靠性**：随机特征不是“启发式技巧”，而是有严格误差界支撑的核函数逼近方法。

### 借助随机特征：核回归向线性回归的转化
核岭回归原本是高复杂度的核方法，但通过随机特征的逼近，能转化为高效的线性回归模型，实现规模化应用。


#### 原始核岭回归的形式与瓶颈
核岭回归的模型形式为：
$$f(x) = \sum_{i=1}^N \alpha_i k(x, x_i) \tag{9.47}$$
它需要求解一个$N \times N$规模的系统（$N$是训练样本数）。

这带来了核心瓶颈：训练时的矩阵运算复杂度达$O(N^3)$，内存占用达$O(N^2)$——在大数据集上完全不可行。


#### 随机特征的转化：从核回归到线性回归
通过随机特征嵌入$z(x)$（$M$维有限特征），核岭回归可**近似为线性回归模型**：
$$f(x) \approx w^\top z(x)$$

此时的训练目标转化为求解线性回归的正则化最小二乘问题：
$$w^* = \arg \min_{w} \sum_{i=1}^N \left| w^\top z(x_i) - y_i \right|^2 + \lambda \|w\|^2 \tag{9.48}$$
（$\lambda$是正则化参数，避免过拟合）


随机特征的介入彻底解决了核回归的规模化问题，具体优势包括：
- **计算成本骤降**：训练复杂度从$O(N^3)$降至$O(NM^2)$（$M$是特征维度，通常$M \ll N$）；
- **内存压力缓解**：存储需求从$O(N^2)$（核矩阵）降至$O(NM)$（随机特征矩阵）；
- **适配大规模优化器**：可结合SGD、Adam等随机优化算法，轻松处理百万级甚至更大规模的数据集；
- **场景拓展性强**：可作为“随机试验空间”应用于偏微分方程（PDE）求解等领域。

随机特征的本质是**用有限维线性模型近似无穷维核方法**，既保留了核方法的表达能力，又获得了线性模型的高效性——这让核方法真正具备了“神经网络级别的规模化能力”。


### 深度随机神经网络（RaNNs）：隐藏层随机化的高效深度模型
深度随机神经网络（Randomized Neural Networks，简称RaNNs）是一类**隐藏层参数随机化、仅训练输出层**的深度模型，既保留了深度网络的表达能力，又大幅降低了训练成本。


#### RaNNs的架构形式
典型的RaNNs由$L$层构成，前$L-1$层为随机化隐藏层，最后一层为可训练的输出层：
$$
\begin{align*}
h_1(x) &= \sigma(W_1x + b_1), \\
h_2(x) &= \sigma(W_2h_1 + b_2), \\
&\vdots \\
u(x) &= W_L h_{L-1}(x)
\end{align*} \tag{9.49}
$$
其中$\sigma$是ReLU、sigmoid等非线性激活函数，$h_\ell$表示第$\ell$层的隐藏输出。


RaNNs的关键设计是**隐藏层参数随机化**，具体规则为：
- **隐藏层参数随机生成**：前$L-1$层的权重$W_1, \dots, W_{L-1}$与偏置$b_\ell$，均从预设分布（如高斯分布、均匀分布）中随机采样，训练过程中保持固定；
- **仅训练输出层**：只有最后一层的权重$W_L$是可学习参数；
- **训练转化为凸优化**：由于隐藏层输出固定，训练等价于求解**凸最小二乘问题**：
  $$W_L^* = \arg \min_{W} \|WH - y\|_2^2 \tag{9.50}$$
  其中$H = [h_1(x_i), \dots, h_{L-1}(x_i)]$是所有训练样本的隐藏层输出构成的矩阵。



RaNNs的工作逻辑可以理解为：
随机化的隐藏层相当于生成一个**大规模、高表达性的非线性特征字典**（每个隐藏神经元对应一个“字典元素”），而输出层的作用是从这个字典中选择“元素”，并学习最优的线性组合以拟合目标。


深度随机神经网络的核心是“**深度结构提供非线性表达，随机化隐藏层降低训练复杂度，顶层线性回归保证训练高效**”——即：
$$\text{RaNNs} = \text{隐藏层随机化的深度网络} + \text{顶层线性回归} \tag{9.51}$$



### 理论视角：深度随机神经网络（RaNNs）是深度核方法的有限逼近
从核方法的角度看，深度随机神经网络（RaNNs）的本质是**深度核的蒙特卡罗近似**——随机化隐藏层通过递归变换诱导出“隐式深度核”，而RaNN的输出层则是对该核的线性拟合。


#### 隐藏层诱导的隐式核函数
RaNN的前$L-1$层随机隐藏层，会在输出层前的隐藏表示$h_{L-1}(x)$上，诱导出一个**隐式核函数**：
$$k_L(x, x') = h_{L-1}(x)^\top h_{L-1}(x')\tag{9.52}$$

这个核的生成是**逐层递归的**：每一层的核函数由上一层的核函数经随机变换得到，具体形式为：
$$k_{\ell+1}(x, x') = \mathbb{E}_{W_\ell, b_\ell} \left[ \sigma(W_\ell h_\ell(x)) \sigma(W_\ell h_\ell(x')) \right]\tag{9.53}$$

其中：
- $W_\ell, b_\ell$是第$\ell$层的随机参数（期望是对其采样分布的平均）；
- $\sigma$是激活函数，它让核函数从“线性”向“非线性复合”演化。


#### 推论：RaNN与深度核的关联
从隐式核的视角，RaNN的特性可总结为：
1. **RaNN是深度核的有限维逼近**：
   RaNN的隐藏层输出$h_{L-1}(x)$，等价于深度核对应的“有限维随机特征映射”——因此RaNN可看作深度核方法的蒙特卡罗近似。

2. **层数增加对应核的复合**：
   每增加一层，核函数会经过一次$\mathcal{T}_\sigma$（激活函数诱导的变换算子）的复合，即：
   $$k_L = \mathcal{T}_\sigma(\mathcal{T}_\sigma(\dots \mathcal{T}_\sigma(k_0) \dots))\tag{9.54}$$
   （$k_0$是初始核，通常为线性核），这让核函数的表达能力随层数增加而增强。

3. **与神经正切核（NTK）关联**：
   当RaNN的隐藏层宽度趋于无穷时，其诱导核会收敛到对应的神经正切核（NTK）——这建立了RaNN与深度网络理论极限的联系。

4. **在特定函数空间中收敛**：
   从逼近理论看，RaNN能在Barron空间（刻画深度网络表达能力的函数空间）或复合函数类中收敛到目标函数，保证了其表达能力的理论边界。


深度RaNN的强表达能力，并非来自“训练复杂的深度参数”，而是来自“随机隐藏层诱导的深度核”——它是**深度核的蒙特卡罗近似**，既保留了深度核的非线性表达能力，又通过“随机特征+线性输出层”的结构避免了深度网络的优化难题。